# Imports

In [1]:
from accelerate import Accelerator
from diffusers import DiffusionPipeline

/home/shane/projects/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Parameters

In [2]:
low_mem = True
#MODEL_NAME = "stabilityai/stable-diffusion-2-1"
MODEL_NAME = "CompVis/stable-diffusion-v1-4"
INSTANCE_DIR = "./content/data/sks"
CLASS_NAME = "dog"
CLASS_DIR = f"./content/data/{CLASS_NAME}"
TOKEN_NAME = "sks"
OUTPUT_DIR = "./content/stable_diffusion_weights/sks"
PRIOR_LOSS_WEIGHT = 1.0
RESOLUTION = 512
TRAIN_BATCH_SIZE = 1
GRAD_ACCUMULATION_STEPS = 1
LEARNING_RATE = 5e-6
LR_SCHEDULER = "constant"
LR_WARMUP_STEPS = 0
NUM_CLASS_IMAGES = 200
MAX_TRAIN_STEPS = 800
SAMPLE_BATCH_SIZE = 1

# Prep host

In [3]:
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

NVIDIA GeForce RTX 2070 SUPER, 8192 MiB, 7674 MiB


In [4]:
#!wget -q https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth.py
#!wget -q https://raw.githubusercontent.com/huggingface/diffusers/main/examples/text_to_image/train_text_to_image.py
#!pip install git+https://github.com/huggingface/diffusers
#!pip install -U -r https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/requirements.txt

In [6]:
#!mkdir -p $INSTANCE_DIR
!mkdir -p $OUTPUT_DIR

# Training

In [7]:
args = [
    "--pretrained_model_name_or_path="+MODEL_NAME,
    "--instance_data_dir="+INSTANCE_DIR,
    "--class_data_dir="+CLASS_DIR,
    "--output_dir="+OUTPUT_DIR,
    "--with_prior_preservation",
    "--prior_loss_weight="+str(PRIOR_LOSS_WEIGHT),
    "--instance_prompt='photo of "+TOKEN_NAME + " " + CLASS_NAME + "'",
    "--class_prompt='photo of " + CLASS_NAME + "'",
    "--resolution="+str(RESOLUTION),
    "--train_batch_size="+str(TRAIN_BATCH_SIZE),
    "--gradient_accumulation_steps="+str(GRAD_ACCUMULATION_STEPS),
    "--learning_rate="+str(LEARNING_RATE),
    "--lr_scheduler="+str(LR_SCHEDULER),
    "--lr_warmup_steps="+str(LR_WARMUP_STEPS),
    "--num_class_images="+str(NUM_CLASS_IMAGES),
    "--max_train_steps="+str(MAX_TRAIN_STEPS),
]
if low_mem:
    args += [
        "--sample_batch_size="+str(SAMPLE_BATCH_SIZE),
        "--mixed_precision='fp16'",
        "--enable_xformers_memory_efficient_attention",
        "--set_grads_to_none",
        "--use_8bit_adam"
    ]
elif not low_mem:
    args += [
        "--train_text_encoder"
    ]
train_args = " ".join(f'{w}' for w in args)

In [8]:
!accelerate launch train_dreambooth.py {train_args}

/home/shane/projects/.venv/lib/python3.10/site-packages/accelerate/accelerator.py:249: FutureWarning: `logging_dir` is deprecated and will be removed in version 0.18.0 of 🤗 Accelerate. Use `project_dir` instead.
  warnings.warn(
04/01/2023 12:58:40 - INFO - __main__ - Distributed environment: DEEPSPEED
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda:0

Mixed precision type: fp16
ds_config: {'train_batch_size': 'auto', 'train_micro_batch_size_per_gpu': 'auto', 'gradient_accumulation_steps': 1, 'zero_optimization': {'stage': 2, 'offload_optimizer': {'device': 'cpu'}, 'offload_param': {'device': 'cpu'}, 'stage3_gather_16bit_weights_on_model_save': False}, 'steps_per_print': inf, 'fp16': {'enabled': True, 'auto_cast': True}, 'bf16': {'enabled': False}}

You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
{'clip_sample_range', 'thresholding', 'variance_type', 'dynami

# Convert to inference pipeline

In [36]:
# Load the pipeline with the same arguments (model, revision) that were used for training
pipeline = DiffusionPipeline.from_pretrained(MODEL_NAME)
accelerator = Accelerator()

# Use text_encoder if `--train_text_encoder` was used for the initial training
if low_mem:
    unet = accelerator.prepare(pipeline.unet)
else: 
    unet, text_encoder = accelerator.prepare(pipeline.unet, pipeline.text_encoder)

# Restore state from a checkpoint path. You have to use the absolute path here.
accelerator.load_state(OUTPUT_DIR + "/checkpoint-500/pytorch_model")

# Rebuild the pipeline with the unwrapped models (assignment to .unet and .text_encoder should work too)
if low_mem:
    pipeline = DiffusionPipeline.from_pretrained(MODEL_NAME, unet=accelerator.unwrap_model(unet))
else: 
    pipeline = DiffusionPipeline.from_pretrained(
        MODEL_NAME,
        unet=accelerator.unwrap_model(unet),
        text_encoder=accelerator.unwrap_model(text_encoder),
    )

# Perform inference, or save, or push to the hub
pipeline.save_pretrained("dreambooth-pipeline")

/home/shane/projects/.venv/lib/python3.10/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
/home/shane/projects/.venv/lib/python3.10/site-packages/safetensors/torch.py:99: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(filename, framework="pt", device=device) as f:
/home/shane/projects/.venv/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStora

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:12                                                                                   │
│                                                                                                  │
│    9 │   unet, text_encoder = accelerator.prepare(pipeline.unet, pipeline.text_encoder)          │
│   10                                                                                             │
│   11 # Restore state from a checkpoint path. You have to use the absolute path here.             │
│ ❱ 12 accelerator.load_state(OUTPUT_DIR + "/checkpoint-500/pytorch_model")                        │
│   13                                                                                             │
│   14 # Rebuild the pipeline with the unwrapped models (assignment to .unet and .text_encoder     │
│   15 if low_mem:                                                                                 │
│                                                                                                  │
│ /home/shane/projects/.venv/lib/python3.10/site-packages/accelerate/accelerator.py:2396 in        │
│ load_state                                                                                       │
│                                                                                                  │
│   2393 │   │   for hook in self._load_model_state_pre_hook.values():                             │
│   2394 │   │   │   hook(models, input_dir)                                                       │
│   2395 │   │                                                                                     │
│ ❱ 2396 │   │   load_accelerator_state(                                                           │
│   2397 │   │   │   input_dir, models, optimizers, schedulers, self.state.process_index, self.sc  │
│   2398 │   │   )                                                                                 │
│   2399 │   │   custom_checkpoints = [f for f in os.listdir(input_dir) if "custom_checkpoint" in  │
│                                                                                                  │
│ /home/shane/projects/.venv/lib/python3.10/site-packages/accelerate/checkpointing.py:138 in       │
│ load_accelerator_state                                                                           │
│                                                                                                  │
│   135 │   for i, model in enumerate(models):                                                     │
│   136 │   │   weights_name = f"{MODEL_NAME}.bin" if i == 0 else f"{MODEL_NAME}_{i}.bin"          │
│   137 │   │   input_model_file = os.path.join(input_dir, weights_name)                           │
│ ❱ 138 │   │   models[i].load_state_dict(torch.load(input_model_file, map_location="cpu"), **lo   │
│   139 │   logger.info("All model weights loaded successfully")                                   │
│   140 │                                                                                          │
│   141 │   # Optimizer states                                                                     │
│                                                                                                  │
│ /home/shane/projects/.venv/lib/python3.10/site-packages/torch/serialization.py:791 in load       │
│                                                                                                  │
│    788 │   if 'encoding' not in pickle_load_args.keys():                                         │
│    789 │   │   pickle_load_args['encoding'] = 'utf-8'                                            │
│    790 │                                                                                         │
│ ❱  791 │   with _open_file_like(f, 'rb') as opened_file:                                         │
│    792 │   │   if _is_zipfile(opened_file):                